# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** B

**Names:**

* Vincenzo Bazzucchi
* Amaury Combes
* Alexis Montavon

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.2 Network sampling

#### Exercise 2.7: Random walk on the Facebook network

In [1]:
# ... WRITE YOUR CODE HERE..
import requests
import numpy as np
from random import choice
URL_TEMPLATE = 'http://iccluster118.iccluster.epfl.ch:5050/v1.0/facebook?user={user_id}'
get_user_data = lambda id: requests.get(URL_TEMPLATE.format(user_id=id)).json()  

In [2]:
def random_walk(source, n):
    ages = np.zeros(n)
    u = source
    for i in range(n):
        u_data = get_user_data(u)
        ages[i] = u_data["age"]
        u = choice(u_data["friends"])
    return np.sum(ages) / len(ages)

In [3]:
source = "f30ff3966f16ed62f5165a229a19b319"
average_age = random_walk(source, 100) # 100 random for now, as I cannot execute.

#### Exercise 2.8